In [97]:
import xarray as xr
import numpy as np
import pandas as pd
import zarr

## Prepare data

In [98]:
infile = '/g/data/xv83/dbi599/precip_c5-d60-pX-f6_19861101-19871101_atmos_isobaric_daily.zarr.zip'

In [99]:
ds = xr.open_zarr(infile)
#with zarr.ZipStore(infile, mode='r') as store:
#    ds = xr.open_zarr(store)

In [100]:
type(ds)

xarray.core.dataset.Dataset

#### Convert units

In [101]:
pr_daily = ds['precip'] * 86400
pr_daily.attrs['units'] = 'mm'
pr_daily

<xarray.DataArray 'precip' (init_date: 4, lead_time: 3653, ensemble: 96, lat: 19, lon: 19)>
dask.array<mul, shape=(4, 3653, 96, 19, 19), dtype=float64, chunksize=(1, 28, 96, 19, 19), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(3653, 4), meta=np.ndarray>
Attributes:
    units:    mm

## Annual rainfall

#### Switch lead_time for time (so annual totals can be calculated)

In [102]:
def reindex_forecast(ds, dropna=False):
    """Switch out lead_time axis for time axis (or vice versa) in a forecast dataset."""
    
    if 'lead_time' in ds.dims:
        index_dim = 'lead_time'
        reindex_dim = 'time'
    elif 'time' in ds.dims:
        index_dim = 'time'
        reindex_dim = 'lead_time'
    else:
        raise ValueError("Neither a time nor lead_time dimension can be found")
    swap = {index_dim: reindex_dim}
    to_concat = []
    for init_date in ds['init_date']:
        fcst = ds.sel({'init_date': init_date})
        fcst = fcst.where(fcst[reindex_dim].notnull(), drop=True)
        fcst = fcst.assign_coords({'lead_time': fcst['lead_time'].astype(int)})
        to_concat.append(fcst.swap_dims(swap))
    concat = xr.concat(to_concat, dim='init_date')
    if dropna:
        return concat.where(concat.notnull(), drop=True)
    else:
        return concat

In [103]:
pr_daily_reindexed = reindex_forecast(pr_daily)
pr_daily_reindexed

,Array,Chunk
Bytes,5.27 GB,7.76 MB
Shape,"(4, 4748, 96, 19, 19)","(1, 28, 96, 19, 19)"
Count,14113 Tasks,2084 Chunks
Type,float64,numpy.ndarray


#### Calculate annual totals

In [104]:
def sum_min_samples(ds, dim, min_samples):
    """Return sum only if there are more than min_samples along dim."""
    
    s = ds.sum(dim, skipna=False)
    # Reference to final lead in sample
    if 'lead_time' in ds.coords:
        if dim in ds['lead_time'].dims:
            l = ds['lead_time'].max(dim, skipna=False)
            s = s.assign_coords({'lead_time': l if len(ds[dim]) >= min_samples else np.nan*l})
    if 'lead_year' in ds.coords:
        l = ds['lead_year'].max(dim, skipna=False)
        s = s.assign_coords({'lead_year': l if len(ds[dim]) >= min_samples else np.nan*l})
        
    return s if len(ds[dim]) >= min_samples else np.nan*s

In [105]:
# total annual (Dec- Nov) precip (mm)
pr_annual = pr_daily_reindexed.resample(time="A-DEC", label='right').apply(sum_min_samples, dim='time', min_samples=365)

In [106]:
pr_annual

<xarray.DataArray 'precip' (init_date: 4, time: 14, ensemble: 96, lat: 19, lon: 19)>
dask.array<transpose, shape=(4, 14, 96, 19, 19), dtype=float64, chunksize=(1, 1, 96, 19, 19), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1984-12-31 1985-12-31 ... 1997-12-31
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    lead_time  (time, init_date) float64 nan nan nan nan ... nan nan nan nan

In [107]:
pr_annual['time']

<xarray.DataArray 'time' (time: 14)>
array(['1984-12-31T00:00:00.000000000', '1985-12-31T00:00:00.000000000',
       '1986-12-31T00:00:00.000000000', '1987-12-31T00:00:00.000000000',
       '1988-12-31T00:00:00.000000000', '1989-12-31T00:00:00.000000000',
       '1990-12-31T00:00:00.000000000', '1991-12-31T00:00:00.000000000',
       '1992-12-31T00:00:00.000000000', '1993-12-31T00:00:00.000000000',
       '1994-12-31T00:00:00.000000000', '1995-12-31T00:00:00.000000000',
       '1996-12-31T00:00:00.000000000', '1997-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1984-12-31 1985-12-31 ... 1997-12-31

In [108]:
pr_annual['lead_time']

<xarray.DataArray 'lead_time' (time: 14, init_date: 4)>
array([[  nan,   nan,   nan,   nan],
       [ 425.,   nan,   nan,   nan],
       [ 790.,  425.,   nan,   nan],
       [1155.,  790.,  425.,   nan],
       [1521., 1156.,  791.,  426.],
       [1886., 1521., 1156.,  791.],
       [2251., 1886., 1521., 1156.],
       [2616., 2251., 1886., 1521.],
       [2982., 2617., 2252., 1887.],
       [3347., 2982., 2617., 2252.],
       [  nan, 3347., 2982., 2617.],
       [  nan,   nan, 3347., 2982.],
       [  nan,   nan,   nan, 3348.],
       [  nan,   nan,   nan,   nan]])
Coordinates:
  * time       (time) datetime64[ns] 1984-12-31 1985-12-31 ... 1997-12-31
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01
    lead_time  (time, init_date) float64 nan nan nan nan ... nan nan nan nan

Convert lead times from days since init_date to years since init_date... 

In [109]:
DAYS_TO_ZERO_LEAD = 60   # what does this mean? 
lead_time_years = np.floor(((pr_annual.lead_time - DAYS_TO_ZERO_LEAD) / 365))
pr_annual = pr_annual.assign_coords({'lead_time': lead_time_years})
pr_annual['lead_time'].attrs['units'] = 'A'
pr_annual['lead_time']

<xarray.DataArray 'lead_time' (time: 14, init_date: 4)>
array([[nan, nan, nan, nan],
       [ 1., nan, nan, nan],
       [ 2.,  1., nan, nan],
       [ 3.,  2.,  1., nan],
       [ 4.,  3.,  2.,  1.],
       [ 5.,  4.,  3.,  2.],
       [ 6.,  5.,  4.,  3.],
       [ 7.,  6.,  5.,  4.],
       [ 8.,  7.,  6.,  5.],
       [ 9.,  8.,  7.,  6.],
       [nan,  9.,  8.,  7.],
       [nan, nan,  9.,  8.],
       [nan, nan, nan,  9.],
       [nan, nan, nan, nan]])
Coordinates:
  * time       (time) datetime64[ns] 1984-12-31 1985-12-31 ... 1997-12-31
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01
    lead_time  (time, init_date) float64 nan nan nan nan 1.0 ... nan nan nan nan
Attributes:
    units:    A

In [118]:
pr_annual['init_date']

<xarray.DataArray 'init_date' (init_date: 4)>
array(['1984-11-01T00:00:00.000000000', '1985-11-01T00:00:00.000000000',
       '1986-11-01T00:00:00.000000000', '1987-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01

#### Spatial averaging

In [110]:
# spatial average of the total annual rainfall

# (Only some notebooks do spatial averaging. Others don't.)

# local_funcs.py used to have a function for this, which applied a region mask.
# That function doesn't exist anymore and I'm not sure how to access the region mask file
#
#def average_region(ds, region_mask):
#    return ds.where(region_mask).mean(['lat','lon'])

#da = da.mean(['lat', 'lon'])
#da

Switch back to lead_time index instead of time...

In [111]:
pr_annual = reindex_forecast(pr_annual)
pr_annual

<xarray.DataArray 'precip' (init_date: 4, lead_time: 9, ensemble: 96, lat: 19, lon: 19)>
dask.array<concatenate, shape=(4, 9, 96, 19, 19), dtype=float64, chunksize=(1, 1, 96, 19, 19), chunktype=numpy.ndarray>
Coordinates:
    time       (init_date, lead_time) datetime64[ns] 1985-12-31 ... 1996-12-31
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1984-11-01 1985-11-01 ... 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
  * lead_time  (lead_time) int64 1 2 3 4 5 6 7 8 9

## Do the bias correction

In [89]:
def stack_by_init_date(da, init_dates, N_lead_steps, init_date_name='init_date', 
                       lead_time_name='lead_time', time_name='time'):
    """ 
        Stacks provided timeseries array in an inital date / lead time format. Note this process \
                replicates data and can substantially increase memory usage. Lead time frequency will match \
                frequency of input data. Returns nans if requested times lie outside of the available range
        
        | Author: Dougie Squire
        | Date: 14/03/2018
        
        Parameters
        ----------
        da : xarray DataArray
            Timeseries array to be stacked
        init_dates : array_like of datetime objects
            Initial dates to stack onto
        N_lead_steps : value
            Number of lead time steps
        init_date_name : str, optional
            Name of initial date dimension
        lead_time_name : str, optional
            Name of lead time dimension
        time_name : str, optional
            Name of time dimension
            
        Returns
        -------
        stacked : xarray DataArray
            Stacked xarray in inital date / lead time format
            
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(3)), 
        ...                  coords=[('time', pd.date_range(start='2000-01-01', periods=3, freq='MS'))])
        >>> init_dates = pd.date_range(start='1999-11-01', periods=3, freq='MS')
        >>> doppyo.utils.stack_by_init_date(A, init_dates=init_dates, N_lead_steps=3)
        <xarray.DataArray (init_date: 3, lead_time: 3)>
        array([[      nan,       nan,       nan],
               [      nan,       nan,       nan],
               [ 0.509276, -3.046124, -0.665343]])
        Coordinates:
          * lead_time  (lead_time) int64 0 1 2
          * init_date  (init_date) datetime64[ns] 1999-11-01 1999-12-01 2000-01-01
    """

    if da[time_name].ndim == 1:
        time_values = da[time_name].values
    else:
        time_values = da[time_name][0, :].values
    freq = pd.infer_freq(time_values)
    if freq is None:
        raise ValueError('Cannot determine frequency of input timeseries')
    
    init_list = []
    for init_date in init_dates:
        start_index = np.where(da[time_name] == np.datetime64(init_date))[0]

        if start_index.size == 0:
            end_time = np.datetime64(pd.DatetimeIndex([init_date]).shift(N_lead_steps, freq=freq)[0])
            end_index = np.where(da[time_name] == end_time)[0]
            if end_index.size == 0:
                # Try step back from the end time - this can help with strange frequencies eg Q-DEC
                start_time = np.datetime64(pd.DatetimeIndex([end_time]).shift(-N_lead_steps, freq=freq)[0])
                start_index = np.where(da[time_name] == start_time)[0]
                
                if start_index.size == 0:
                    da_slice = da.isel({time_name:range(0, N_lead_steps)}).where(False, np.nan)
                    time_slice = da_slice[time_name].where(False, np.datetime64('nat')) \
                                                    .expand_dims({init_date_name: [init_date]})
                    init_list.append(da_slice.expand_dims({init_date_name: [init_date]}) \
                                             .assign_coords({f'{time_name}_new': time_slice}) \
                                             .assign_coords({time_name: np.arange(0, N_lead_steps)}) \
                                             .rename({time_name: lead_time_name}) \
                                             .rename({f'{time_name}_new': time_name}))
                else:
                    start_index = start_index.item()
                    da_slice = da.isel({time_name:slice(start_index, None)})
                    time_slice = da_slice[time_name].expand_dims({init_date_name: [init_date]})
                    init_list.append(da_slice.expand_dims({init_date_name: [init_date]}) \
                                             .assign_coords({f'{time_name}_new': time_slice}) \
                                             .assign_coords({time_name: np.arange(0, len(time_slice[time_name]))}) \
                                             .rename({time_name: lead_time_name}) \
                                             .rename({f'{time_name}_new': time_name}))
            else:
                end_index = end_index.item()
                start_index = end_index - N_lead_steps
                if start_index < 0:
                    hang = -start_index
                    start_index = 0
                else: hang = 0
                    
                da_slice = da.isel({time_name:range(start_index, end_index)})
                time_slice = da_slice[time_name].expand_dims({init_date_name: [init_date]})
                init_list.append(da_slice.expand_dims({init_date_name: [init_date]}) \
                                         .assign_coords({f'{time_name}_new': time_slice}) \
                                         .assign_coords({time_name: np.arange(0, end_index-start_index)+hang}) \
                                         .rename({time_name: lead_time_name}) \
                                         .rename({f'{time_name}_new': time_name}))
        else:
            start_index = start_index.item()
            end_index = min([start_index + N_lead_steps, len(da[time_name])])
            
            da_slice = da.isel({time_name:range(start_index, end_index)})
            time_slice = da_slice[time_name].expand_dims({init_date_name: [init_date]})
            init_list.append(da_slice.expand_dims({init_date_name: [init_date]}) \
                                     .assign_coords({f'{time_name}_new': time_slice}) \
                                     .assign_coords({time_name: np.arange(0, end_index-start_index)}) \
                                     .rename({time_name: lead_time_name}) \
                                     .rename({f'{time_name}_new': time_name}))
            
    stacked = xr.concat(init_list, dim=init_date_name)
    stacked[lead_time_name].attrs['units'] = freq
    
    return stacked

In [ ]:
#%%time
#fcst = f5_precip_ann
#obsv = awap_precip_ann
#
## Force leads to integers so fcst can be reindexed
#fcst_hack = lf.reindex_forecast_hack(fcst)
#
#stack_dates = fcst_hack.sel(
#    init_date=slice(
#        (pd.to_datetime(period[0]) - pd.DateOffset(years=10)).strftime('%Y-%m-%d'),
#         pd.to_datetime(period[1]).strftime('%Y-%m-%d'))).init_date.values
#
#obsv_stacked = lf.stack_by_init_date(
#    obsv, stack_dates, max(fcst_hack.lead_time.values)+2) 
## Shift leads back by one so that obsv dates match fcst dates
#obsv_stacked = obsv_stacked.assign_coords({'lead_time': obsv_stacked.lead_time-1}) \
#                           .where(obsv_stacked.lead_time > 0, drop=True)
#
#mask = (fcst_hack.time >= period[0]) & (fcst_hack.time <= period[1])
#    
#fcst_clim = fcst_hack.mean('ensemble').where(mask, drop=True).groupby('init_date.month').mean('init_date')
#obsv_clim = obsv_stacked.where(mask, drop=True).groupby('init_date.month').mean('init_date')
#
#model_mulbias_precip_ann = (fcst_clim / obsv_clim).chunk('auto').rename('precip_ann')
#model_mulbias_precip_ann = model_mulbias_precip_ann.xc.cache(f'CAFE-model-mulbias_precip-ann-accl_{REGION_NAME}.zarr', clobber=clobber)
#
#model_addbias_precip_ann = (fcst_clim - obsv_clim).chunk('auto').rename('precip_ann')
#model_addbias_precip_ann = model_addbias_precip_ann.xc.cache(f'CAFE-model-addbias_precip-ann-accl_{REGION_NAME}.zarr', clobber=clobber)

In [ ]:
#period = (np.datetime64('1990-01-01'), 
#          np.datetime64('2020-12-31'))

# model_mulbias_precip = lf.estimate_model_biases(f5_precip, awap_precip, period, mode='multiplicative').rename('precip')
# model_addbias_precip = lf.estimate_model_biases(f5_precip, awap_precip, period, mode='additive').rename('precip')

#def estimate_model_biases(fcst, obsv, period, mode):
#    """ mode = 'additive' or 'multiplicative' """
#    stack_dates = fcst.sel(
#        init_date=slice(
#            (pd.to_datetime(period[0]) - pd.DateOffset(years=10)).strftime('%Y-%m-%d'),
#            pd.to_datetime(period[1]).strftime('%Y-%m-%d'))).init_date.values
#
#    obsv_stacked = stack_by_init_date(
#        obsv, stack_dates, len(fcst.lead_time))
#    
#    mask = (fcst.time >= period[0]) & (fcst.time <= period[1])
#    
#    fcst_clim = fcst.mean('ensemble').where(mask, drop=True).groupby('init_date.month').mean('init_date')
#    obsv_clim = obsv_stacked.where(mask, drop=True).groupby('init_date.month').mean('init_date')
#    
#    if mode == 'additive':
#        return (fcst_clim - obsv_clim).chunk('auto')
#    elif mode == 'multiplicative':
#        return (fcst_clim / obsv_clim).chunk('auto')
#    else: 
#        raise ValueError(f'Unrecognised mode {mode}')
        

In [ ]:
#f5_precip_mbc = lf.remove_model_bias(f5_precip, model_mulbias_precip, mode='multiplicative').rename('precip')
#f5_precip_abc = lf.remove_model_bias(f5_precip, model_addbias_precip, mode='additive').rename('precip')

#def remove_model_bias(fcst, bias, mode):
#    """ mode = 'additive' or 'multiplicative' """
#    if mode == 'additive':
#        return (fcst.groupby('init_date.month') - bias).chunk(fcst.chunks)
#    elif mode == 'multiplicative':
#        return (fcst.groupby('init_date.month') / bias).chunk(fcst.chunks)
#    else: 
#        raise ValueError(f'Unrecognised mode {mode}')